In [68]:
import pandas as pd

In [69]:

df_preventiva = pd.read_excel('Preventiva_07-01-2026.xlsx')
df_carreta = pd.read_excel('CARRETA.xlsx')
df_esl = pd.read_excel('magazine_2026_01_07_09_40.xlsx')
df_mobile = pd.read_excel('Mobile_07-01-2026.xls')
df_bipe = pd.read_excel('Bipe_Produtos_07-01-2026.xlsx')
df_bipe_notas = pd.read_excel('Bipe_de_notas_07-01-2026.xlsx', sheet_name='Plan1')


In [70]:
dfs = [df_preventiva, df_carreta, df_esl , df_mobile,df_bipe, df_bipe_notas  ]

In [71]:
for col in dfs:
    col.columns = col.columns.str.strip()

In [6]:
df_carreta.columns

Index(['PEDIDO', 'NF`s', 'ROMANEIO/ID', 'TRANSP / MOTO', 'CD/FILIAL',
       'DATA_ROMANEIO', 'DATA ENTRADA', 'HORA', 'PREVISÃO ENTREGA', 'TIPO',
       'PEDIDOS SISTEMA', 'CHAVE', 'OBS', 'UUID'],
      dtype='object')

In [72]:
list_colunas = ['PEDIDO', 'NF`s', 'PREVISÃO ENTREGA', 'TIPO', 'CHAVE']

In [49]:
df_carreta[list_colunas]

,PEDIDO,NF`s,PREVISÃO ENTREGA,TIPO,CHAVE
0,1117092784,237373,2025-12-01,Porta a Porta,52251147960950110733550070002373731034600479
1,1117143808,237379,2025-12-01,Porta a Porta,52251147960950110733550070002373791018718413
2,1117100513,237374,2025-12-01,Porta a Porta,52251147960950110733550070002373741028039326
3,1116937756,237355,2025-12-01,Porta a Porta,52251147960950110733550070002373551020024162
4,1117118870,237382,2025-12-01,Porta a Porta,52251147960950110733550070002373821017920168
...,...,...,...,...,...
8970,1129087542,324584,2026-01-07,Porta a Porta,52260147960950110733550070003245841085309746
8971,1129219343,324655,2026-01-07,Porta a Porta,52260147960950110733550070003246551063138780
8972,1127475847,324551,2026-01-07,Porta a Porta,52260147960950110733550070003245511014143704
8973,1129070316,324573,2026-01-07,Porta a Porta,52260147960950110733550070003245731068801340


In [73]:
df_carreta = df_carreta[list_colunas]

In [10]:
df_mobile.columns

Index(['Pedido Cliente', 'Pedido', 'UUID', 'Romaneio', 'Transportador',
       'Nota Fiscal', 'Numero de Série', 'Data Importação Pedido',
       'Data de Captura', 'Data de Processamento', 'Entregador', 'Tipo'],
      dtype='object')

In [11]:
df_bipe_notas.columns

Index(['NF', 'CHAVE', 'OCORRENCIA', 'CD'], dtype='object')

In [12]:
df_carreta.columns

Index(['PEDIDO', 'NF`s', 'PREVISÃO ENTREGA', 'TIPO', 'CHAVE'], dtype='object')

In [74]:
df_final = (pd.merge(df_carreta, df_esl[['Nota Fiscal/Chave NF-e',
                 'Última ocorrência/Observações',
                 'Pessoa/Nome',
                 'Última ocorrência/Data ocorrência']], left_on='CHAVE', right_on='Nota Fiscal/Chave NF-e',
                 how='left').drop(columns=['Nota Fiscal/Chave NF-e'])
                 .merge(df_mobile[['Pedido','Tipo', 'Entregador' ]], left_on='PEDIDO', right_on='Pedido', how='left').drop(columns=['Pedido'])
                 .merge(df_bipe[['PEDIDOS', 'INFO']], left_on='PEDIDO', right_on='PEDIDOS', how='left').drop(columns=['PEDIDOS'])
                 .merge(df_bipe_notas[['NF','OCORRENCIA']], left_on='NF`s', right_on='NF', how='left').drop(columns=['NF'])
        
                 )

In [75]:
df_final_2 = (pd.merge(df_carreta,df_esl[['Nota Fiscal/Chave NF-e', 'Ocorrência/Ocorrência']], left_on='CHAVE', right_on='Nota Fiscal/Chave NF-e', how='left')
              .drop(columns=['Nota Fiscal/Chave NF-e']))

In [24]:
df_final.head(2)

,PEDIDO,NF`s,PREVISÃO ENTREGA,TIPO,CHAVE,Última ocorrência/Observações,Pessoa/Nome,Última ocorrência/Data ocorrência,Tipo,Entregador,INFO,OCORRENCIA
0,1117092784,237373,2025-12-01,Porta a Porta,52251147960950110733550070002373731034600479,NaN,VALDEVI DOS REIS FRANCISCO (GOIANIA),2025-12-02 23:00:00,Entrega Realizada Normalmente,john Andrew,NaN,NaN
1,1117143808,237379,2025-12-01,Porta a Porta,52251147960950110733550070002373791018718413,NaN,NaN,2025-12-03 20:00:00,Entrega Realizada Normalmente,JHON CLEVER(GOIANIA),NaN,NaN


In [76]:
df_final['Última ocorrência/Observações'] = df_final['Última ocorrência/Observações'].fillna('Não informado')

In [38]:
df_final.columns

Index(['PEDIDO', 'NF`s', 'PREVISÃO ENTREGA', 'TIPO', 'CHAVE',
       'Última ocorrência/Observações', 'Pessoa/Nome',
       'Última ocorrência/Data ocorrência', 'Tipo', 'Entregador', 'INFO',
       'OCORRENCIA'],
      dtype='object')

In [39]:
df_final_2.columns

Index(['PEDIDO', 'NF`s', 'PREVISÃO ENTREGA', 'TIPO', 'CHAVE',
       'Ocorrência/Ocorrência'],
      dtype='object')

In [36]:
df_final.head(2)

,PEDIDO,NF`s,PREVISÃO ENTREGA,TIPO,CHAVE,Última ocorrência/Observações,Pessoa/Nome,Última ocorrência/Data ocorrência,Tipo,Entregador,INFO,OCORRENCIA
0,1117092784,237373,2025-12-01,Porta a Porta,52251147960950110733550070002373731034600479,NaN,VALDEVI DOS REIS FRANCISCO (GOIANIA),2025-12-02 23:00:00,Entrega Realizada Normalmente,john Andrew,NaN,NaN
1,1117143808,237379,2025-12-01,Porta a Porta,52251147960950110733550070002373791018718413,NaN,NaN,2025-12-03 20:00:00,Entrega Realizada Normalmente,JHON CLEVER(GOIANIA),NaN,NaN


In [77]:
df_final_3 = pd.merge(df_final, df_final_2[['CHAVE', 'Ocorrência/Ocorrência']], left_on='CHAVE', right_on='CHAVE', how='left').drop(columns=['CHAVE'])

In [63]:
df_final_3.head(2)

,PEDIDO,NF`s,PREVISÃO ENTREGA,TIPO,Última ocorrência/Observações,Pessoa/Nome,Última ocorrência/Data ocorrência,Tipo,Entregador,INFO,OCORRENCIA,Ocorrência/Ocorrência
0,1117092784,237373,2025-12-01,Porta a Porta,Não informado,VALDEVI DOS REIS FRANCISCO (GOIANIA),2025-12-02 23:00:00,Entrega Realizada Normalmente,john Andrew,NaN,NaN,Entrega Realizada Normalmente
1,1117143808,237379,2025-12-01,Porta a Porta,Não informado,NaN,2025-12-03 20:00:00,Entrega Realizada Normalmente,JHON CLEVER(GOIANIA),NaN,NaN,ENTREGUE


In [78]:
# 1. Cria uma condição (filtro) onde a coluna é 'Não informado'
filtro = df_final_3['Última ocorrência/Observações'] == 'Não informado'

# 2. Nas linhas onde o filtro for verdadeiro, preenche com o valor da outra coluna
df_final_3.loc[filtro, 'Última ocorrência/Observações'] = df_final_3.loc[filtro, 'Ocorrência/Ocorrência']

In [79]:
df_final_3.head(2)

,PEDIDO,NF`s,PREVISÃO ENTREGA,TIPO,Última ocorrência/Observações,Pessoa/Nome,Última ocorrência/Data ocorrência,Tipo,Entregador,INFO,OCORRENCIA,Ocorrência/Ocorrência
0,1117092784,237373,2025-12-01,Porta a Porta,Entrega Realizada Normalmente,VALDEVI DOS REIS FRANCISCO (GOIANIA),2025-12-02 23:00:00,Entrega Realizada Normalmente,john Andrew,NaN,NaN,Entrega Realizada Normalmente
1,1117143808,237379,2025-12-01,Porta a Porta,ENTREGUE,NaN,2025-12-03 20:00:00,Entrega Realizada Normalmente,JHON CLEVER(GOIANIA),NaN,NaN,ENTREGUE


In [80]:
df_final_3.columns

Index(['PEDIDO', 'NF`s', 'PREVISÃO ENTREGA', 'TIPO',
       'Última ocorrência/Observações', 'Pessoa/Nome',
       'Última ocorrência/Data ocorrência', 'Tipo', 'Entregador', 'INFO',
       'OCORRENCIA', 'Ocorrência/Ocorrência'],
      dtype='object')

In [86]:
df_final_3 =  df_final_3.drop(columns=['Ocorrência/Ocorrência'])

In [87]:
df_final_3.head(2)

,PEDIDO,NF`s,PREVISÃO ENTREGA,TIPO,Última ocorrência/Observações,Pessoa/Nome,Última ocorrência/Data ocorrência,Tipo,Entregador,INFO,OCORRENCIA
0,1117092784,237373,2025-12-01,Porta a Porta,Entrega Realizada Normalmente,VALDEVI DOS REIS FRANCISCO (GOIANIA),2025-12-02 23:00:00,Entrega Realizada Normalmente,john Andrew,NaN,NaN
1,1117143808,237379,2025-12-01,Porta a Porta,ENTREGUE,NaN,2025-12-03 20:00:00,Entrega Realizada Normalmente,JHON CLEVER(GOIANIA),NaN,NaN


In [89]:
df_final_3.to_excel('Relatorio.xlsx', sheet_name='Relatorio_01', index=False)